In [3]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

### Pulling down the Listing Data

In [7]:
r = requests.get('https://www.propertypro.ng/property-for-rent?search=&type=&bedroom=&min_price=&max_price=')

soup = BeautifulSoup(r.content, 'html5lib')

In [20]:
listing_divs = soup.select('div[class=single-room-sale\ listings-property]')
print("Number of apartment listings on a page:", len(listing_divs))

Number of apartment listings on a page: 20


In [21]:
listing_divs[0]

<div class="single-room-sale listings-property">
<div class="single-room-img">
<a class="single-room-img-a" href="/property/4-bedroom-detached-duplex-house-for-rent-queen-elizabeth-by-villa-asokoro-abuja-5EDMD"> <img alt="4 bedroom Detached Duplex House for rent Queen Elizabeth by Villa Asokoro Abuja" class="listings-img" src="https://images.propertypro.ng/medium/tastefully-finished-4bedroom-duplex-pGVL3qqyjNbyAC0eWEQB.jpg" title="4 bedroom Detached Duplex House for rent Queen Elizabeth by Villa Asokoro Abuja"/> </a>
<div class="save-heart">
<a href="/profile/add-favorite/578529"><i class="fa fa-heart"></i></a>
</div>
<div class="room-icons">
<span><span>24</span> <img alt="icon" src="/assets/assets/img/property/2f8b655773948c43f77e8c928e94bbe3-img.png"/> </span>
</div>

</div>
<div class="single-room-text">
<a href="/property/4-bedroom-detached-duplex-house-for-rent-queen-elizabeth-by-villa-asokoro-abuja-5EDMD"> <h2 class="listings-property-title">4 BEDROOM DETACHED DUPLEX HOUSE FOR R

In [33]:
listing_divs[0].select('h4')[0].text

'Queen Elizabeth by VillaAsokoro Abuja'

In [49]:
listing_divs[0].select('h3[class*=listings-price]')[0].text.strip()

'₦ 15,000,000'

In [55]:
listing_divs[0].select('div[class*=fur-areea]')[0].text.strip().split('\n')

['4 beds', '4 baths', '5 Toilets']

Now that I've gotten all the relevant information, I'll put all the codes together;